# COVID Infected Recognition using Deep Learning (CNN)

## Introduction towards Dataset

<b>We build a public available SARS-CoV-2 CT scan dataset, containing 1252 CT scans that are positive for SARS-CoV-2 infection (COVID-19) and 1230 CT scans for patients non-infected by SARS-CoV-2, 2482 CT scans in total. These data have been collected from real patients in hospitals from Sao Paulo, Brazil. The aim of this dataset is to encourage the research and development of artificial intelligent methods which are able to identify if a person is infected by SARS-CoV-2 through the analysis of his/her CT scans. As baseline result for this dataset we used an explainable Deep Learning approach (xDNN) which we could achieve an F1 score of 97.31% which is very promising.</b>


### Reading the Images data

In [7]:
# Deep Learning CNN model to recognize whether COVID Infected or Non-COVID Infected

'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''

TrainingImagePath='C:/Users/Niraj Gupta/Desktop/Naveen/Medical Billing Project/SARS-COV-2 Ct-Scan Dataset/Training Dataset'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 2461 images belonging to 2 classes.
Found 2461 images belonging to 2 classes.


{'COVID': 0, 'non-COVID': 1}

### Creating a list of CT Scan with COVID & Non-COVID and classes

In [8]:
'''#################### Creating lookup table for all CT Scan with COVID & Non-COVID ##############################'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the CT Scan with COVID & Non-COVID Images and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the CT Scan with COVID & Non-COVID map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as f:
    pickle.dump(ResultMap, f, pickle.HIGHEST_PROTOCOL)

print("Mapping of CT Scan with COVID & Non-COVID Images and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of CT Scan with COVID & Non-COVID Images and its ID {0: 'COVID', 1: 'non-COVID'}

 The Number of output neurons:  2


### Creating the CNN model

In [9]:
'''######################## Create CNN deep learning model ####################################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution'''
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))


'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

# Output layer
classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

C:\Users\Niraj Gupta\Anaconda_Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Fitting the model

In [10]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit(
                    training_set,
                    steps_per_epoch=5,
                    epochs=60,
                    validation_data=test_set,
                    validation_steps=2)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Epoch 1/60


C:\Users\Niraj Gupta\Anaconda_Python\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 684ms/step - accuracy: 0.5232 - loss: 0.7097 - val_accuracy: 0.5469 - val_loss: 0.7359
Epoch 2/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step - accuracy: 0.5237 - loss: 0.7225 - val_accuracy: 0.4688 - val_loss: 0.6975
Epoch 3/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 325ms/step - accuracy: 0.4535 - loss: 0.7026 - val_accuracy: 0.7188 - val_loss: 0.6866
Epoch 4/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 332ms/step - accuracy: 0.6617 - loss: 0.6856 - val_accuracy: 0.7031 - val_loss: 0.6682
Epoch 5/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 328ms/step - accuracy: 0.5266 - loss: 0.6863 - val_accuracy: 0.7031 - val_loss: 0.6754
Epoch 6/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 313ms/step - accuracy: 0.6303 - loss: 0.6744 - val_accuracy: 0.5312 - val_loss: 0.6695
Epoch 7/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step - accuracy: 0.5905 - loss: 0.6603 - val_accuracy: 0.7344 - val_loss: 0.6353
Epoch 8/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step - accuracy: 0.5552 - loss: 0.6826 - val_accuracy: 0.5469 - val_loss: 0.6453
Epo

C:\Users\Niraj Gupta\Anaconda_Python\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.6438 - loss: 0.5612 - val_accuracy: 0.7031 - val_loss: 0.5258
Epoch 17/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 215ms/step - accuracy: 0.7023 - loss: 0.5716 - val_accuracy: 0.7500 - val_loss: 0.4939
Epoch 18/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.6275 - loss: 0.6504 - val_accuracy: 0.6250 - val_loss: 0.5563
Epoch 19/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 190ms/step - accuracy: 0.5285 - loss: 0.6817 - val_accuracy: 0.6250 - val_loss: 0.6220
Epoch 20/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy: 0.6961 - loss: 0.5925 - val_accuracy: 0.6562 - val_loss: 0.6258
Epoch 21/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - accuracy: 0.7226 - loss: 0.6052 - val_accuracy: 0.6719 - val_loss: 0.6166
Epoch 22/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - accuracy: 0.7920 - loss: 0.5578 - val_accuracy: 0.8281 - val_loss: 0.4931
Epoch 23/60
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step - accuracy: 0.6848 - loss: 0.6002 - val_accuracy: 0.7812 - val_loss: 0.50

### Saving the model

In [11]:
classifier.save("classifier.keras")

### Testing the model on a different CT Scan with COVID & Non-COVID Images

In [12]:
'''########################## Making single predictions ############################'''
import numpy as np
from keras.preprocessing import image

testImage='C:/Users/Niraj Gupta/Desktop/Naveen/Medical Billing Project/SARS-COV-2 Ct-Scan Dataset/Testing Dataset/COVID/Covid (6).png'
test_image=image.load_img(testImage,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
print('### Printing probabilities for each class ###')
print(result, '\n')
print('### Printing original CT Scan with COVID & Non-COVID Images mapping ###')
print(ResultMap, '\n')
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

### Printing probabilities for each class ###
[[1. 0.]] 

### Printing original CT Scan with COVID & Non-COVID Images mapping ###
{0: 'COVID', 1: 'non-COVID'} 

########################################
Prediction is:  COVID
